# Implementatie van een tweelagig neuraal netwerk voor classificatie


In [ ]:
import numpy as np

Implementeer de sigmoïde:

$\sigma(z) = 1/(1+\exp(-z))$

In [ ]:
def sigmoid(Z):
    """ Z: een tensor met een willekeurige vorm
    
        returns: een tensor met dezelfde vorm van Z waarin de logistische functie 
                 werd toegepast op elk element van Z
    """
    ### BEGIN CODE HIER (1 regel)
    ### EINDE CODE HIER

In [ ]:
### TEST CEL: niet wijzigen
np.random.seed(1)
assert sigmoid(0) == 0.5, "Verkeerd in nul"
assert np.linalg.norm(sigmoid(np.zeros(2)) - np.array([0.5,0.5])) < 10**-6, "Verkeerd in nul nul"
Z = np.random.randn(2,3)
S = sigmoid(Z)
assert Z.shape == S.shape, "Verkeerde vorm"
Sinv = np.log(S/(1-S))
assert np.linalg.norm(Z - Sinv) < 10**-6, "Verkeerd in random punt"

Bereken de uitvoer van het netwerk, alsook de tussenliggende activaties.
We nemen aan dat alle lagen de sigmoïde activatiefunctie gebruiken.

Elke laag doet het volgende:

$Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$
en
$A^{[l]} = \sigma(Z^{[l]})$ voor $l \in \{1,2\}$.

In [ ]:
def forward_propagation(X, W1, W2, b1, b2):
    """ Berekent het resultaat van een neuraal netwerk m.b.v. voorwaartse propagatie.
    
        X  : invoer van de vorm (n0, m)
        W1 : eerste gewichten matrix van de vorm (n1, n0)
        W2 : tweede gewichten matrix van de vorm (n2, n1)
        b1 : bias vector van de eerste laag van de vorm (n1, 1)
        b2 : bias vector van de tweede laag van de vorm (n2, 1)
    """
    ### BEGIN CODE HIER (4 regels)

    ### EINDE CODE HIER
    return (A1, A2)

In [ ]:
### TEST CEL: niet wijzigen
W1 = np.array([[0.15,0.2],[0.25, 0.3]])
b1 = np.array([0.35,0.35]).reshape(-1,1)
W2 = np.array([[0.4,0.45],[0.5, 0.55]])
b2 = np.array([0.6,0.6]).reshape(-1,1)
X = np.array([0.05,0.1]).reshape(-1,1)
A1, A2 = forward_propagation(X, W1, W2, b1, b2)
expected1 = np.array([0.59326999, 0.59688438]).reshape(-1, 1)
expected2 = np.array([0.75136507, 0.77292847]).reshape(-1,1)
assert np.linalg.norm(A1 - expected1) < 10**-6, "Verkeerde 1e laag voor 1 voorbeeld"
assert np.linalg.norm(A2 - expected2) < 10**-6, "Verkeerde 2e laag voor 1 voorbeeld"
X = np.hstack([X, X, X])
expected1 = np.hstack([expected1, expected1, expected1])
expected2 = np.hstack([expected2, expected2, expected2])
A1, A2 = forward_propagation(X, W1, W2, b1, b2)
assert np.linalg.norm(A1 - expected1) < 10**-6, "Verkeerde 1e laag voor 3 voorbeelden"
assert np.linalg.norm(A2 - expected2) < 10**-6, "Verkeerde 2e laag voor 3 voorbeelden"

We schrijven een functie om de kost te berekenen.

$J = 
-\frac{1}{m}\sum_{i=1}^m \sum_{k=1}^K
y^{(i)}_k \ln\bigl(h(x^{(i)})_k\bigr) + (1-y^{(i)}_k)
\ln\bigl(1-h(x^{(i)})_k\bigr).$

In de invoer geeft <code>Yhat</code> de waarde van de hypothese aan.  Het is gemakkelijk 
om een een numpy array te bepalen met

$ y^{(i)}_k \ln\bigl(h(x^{(i)})_k\bigr) + (1-y^{(i)}_k)
\ln\bigl(1-h(x^{(i)})_k\bigr)$ 
op de plaats $(k, i)$. 

Vervolgens sommeer je van deze matrix 
de kolommen (en bekom je een array van de vorm $(1,m)$). Tenslotte bepaal je het gemiddelde
van de elementen in deze array en vermenigvuldig je met $-1$.

In [ ]:
def cost(Yhat, Y):
    assert Yhat.shape == Y.shape
    ### BEGIN CODE HIER (1 of 2 regels)
    ### EINDE CODE HIER
    cost = np.squeeze(cost)  ## Zorg ervoor dat de uitvoer als getal wordt teruggegeven
    return cost

In [ ]:
## TEST Cel: Niet wijzigen
X = np.array([0.05,0.1]).reshape(-1,1)
Y = np.array([0,1]).reshape(-1, 1)
Yhat = np.array([0.7514, 0.7729]).reshape(-1,1)
c = cost(Yhat, Y)
assert c.shape == (), "Kost is geen scalair"
assert abs(c-1.6495157) < 10**-6, "Verkeerde kost"
Yhat  = np.hstack([Yhat, Yhat, Yhat])
Y = np.hstack([Y, Y, Y])
c = cost(Yhat, Y)
assert c.shape == (), "Kost is geen scalair, meerdere voorbeelden"
assert abs(c-1.6495157) < 10**-6, "Verkeerde kost, meerdere voorbeelden"

We implementeren het algoritme voor achterwaartse propagatie waarbij de 
aanpassingsmatrices en aanpassingsvectoren worden bepaald.
In de cursus staat hoe dit te doen voor één enkel voorbeeld:
\begin{equation}\label{eq:backprop}
\left\{
\begin{aligned}
\mathrm{d}z^{[2]} & \gets a^{[2]} - y \\
\mathrm{d}W^{[2]} & \gets \mathrm{d}{z^{[2]}} {a^{[1]}}^T \\
\mathrm{d}b^{[2]} & \gets \mathrm{d}z^{[2]} \\
\mathrm{d}z^{[1]} & \gets {W^{[2]}}^T  \mathrm{d}z^{[2]} \odot a^{[1]}\odot(1-a^{[1]}) \\
\mathrm{d}W^{[1]} & \gets \mathrm{d}z^{[1]} a^{[0]^T} \\
\mathrm{d}b^{[1]} & \gets \mathrm{d}z^{[1]} \\
\end{aligned}
\right.
\end{equation}

Het is echter mogelijk om dit ook te gaan vectoriseren en de berekeningen uit te voeren voor
een aantal voorbeelden tegelijkertijd.  We krijgen:
\begin{equation}
\left\{
\begin{aligned}
\mathrm{d}Z^{[2]} & \gets A^{[2]} - Y                                                & (n^{[2]}, m)\\
\mathrm{d}W^{[2]} & \gets \frac{1}{m}\mathrm{d}{Z^{[2]}} {A^{[1]}}^T                 & (n^{[2]}, n^{[1]})\\
\mathrm{d}b^{[2]} & \gets \frac{1}{m}\mathrm{d}Z^{[2]} \mathbb 1_{m\times1}                              & (n^{[2]}, 1)\\
\mathrm{d}Z^{[1]} & \gets {W^{[2]}}^T  \mathrm{d}Z^{[2]} \odot A^{[1]}\odot(1-A^{[1]})  & (n^{[1]}, m)\\
\mathrm{d}W^{[1]} & \gets \frac{1}{m}\mathrm{d}Z^{[1]} A^{[0]^T}               & (n^{[1]}, n^{[0]}) \\
\mathrm{d}b^{[1]} & \gets \frac{1}{m}\mathrm{d}Z^{[1]}  \mathbb 1_{m\times1}    &    (n^{[1]}, 1)\\
\end{aligned}
\right.
\end{equation}

Merk op dat de matrixvermenigvuldiging met $\mathrm{d}Z \mathbb 1_{m\times 1}$ neerkomt op het nemen van de som van elke 
rij van $\mathrm{d}Z$.

In [ ]:
def backward_propagation(X, Y, W1, W2, b1, b2, A1, A2):
    """ Bepaal de aanpassingsmatrices en aanpassingsvectoren.
    
        X : invoer voor het neuraal netwerk van de vorm (n0, m)
        Y : gewenste uitvoer van de vorm (n2, m)
        W1 : eerste gewichten matrix van de vorm (n1, n0)
        W2 : tweede gewichten matrix van de vorm (n2, n1)
        b1 : eerste bias vector van de vorm (n1, 1)
        b2 : tweede bias vector van de vorm (n2, 1)
        
        returns: de aanpassingmatrices en aanpassingsvectoren: dW1, dW2, db1, db2
    """
    
    (n, m) = X.shape
    ### BEGIN CODE HIER (6 regels)
   
    ### EINDE CODE HIER
    return dW1, dW2, db1, db2

In [ ]:
### TEST CEL: niet wijzigen
X = np.array([0.05,0.1]).reshape(-1,1)
Y = np.array([0,1]).reshape(-1, 1)
W1 = np.array([[0.15,0.2],[0.25, 0.3]])
b1 = np.array([0.35,0.35]).reshape(-1,1)
W2 = np.array([[0.4,0.45],[0.5, 0.55]])
b2 = np.array([0.6,0.6]).reshape(-1,1)
A1, A2 = forward_propagation(X, W1, W2, b1, b2)
dW1, dW2, db1, db2 = backward_propagation(X, Y, W1, W2, b1, b2, A1, A2)
assert W1.shape == dW1.shape, "Verkeerde vorm W1"
assert W2.shape == dW2.shape, "Verkeerde vorm W2"
assert b1.shape == db1.shape, "Verkeerde vorm b1"
assert b2.shape == db2.shape, "Verkeerde vorm b2"
dW2_exp = np.array([[0.4458, 0.4485],[-0.1347, -0.1356]])
db2_exp = np.array([0.7514, -0.2271]).reshape(-1,1)
dW1_exp = np.array([[0.002256, 0.004512],[0.002565, 0.005130]])
db1_exp = np.array([0.04512, 0.05130]).reshape(-1, 1)
assert np.linalg.norm(dW2 - dW2_exp) < 10**-4, "Verkeerde dW2"
assert np.linalg.norm(dW1 - dW1_exp) < 10**-6, "Verkeerde dW1"
assert np.linalg.norm(db2 - db2_exp) < 10**-4, "Verkeerde db2"
assert np.linalg.norm(db1 - db1_exp) < 10**-5, "Verkeerde db1"

De onderstaande twee functies berekenen samen de gradiënt van de kostfunctie op een alternatieve maar veel tragere manier.
Elke paramter wordt lichtjes geperturbeerd en het effect hiervan op de waarde van de kostfunctie wordt bekeken.
Op die manier kan de gradiënt numeriek worden bepaald.

Het doel van deze trage (en eenvoudigere) implementatie is om te verifiëren dat de functie <code>backward_propagation</code> correct werkt.

Je hoeft hier zelf niets te implementeren maar je zou wel moeten snappen wat de code doet.

In [ ]:
def compute_gradient_slow(params, episolon, X, Y, W1, W2, b1, b2, epsilon=10**-6):
    dp = np.zeros_like(params)
    (rows, cols) = params.shape
    for row in range(rows):
        for col in range(cols):
            backup = params[row, col]
            params[row, col] += epsilon
            _, Yhat_plus = forward_propagation(X, W1, W2, b1, b2)
            cost_plus = cost(Yhat_plus, Y)
            params[row, col] -= 2*epsilon
            _, Yhat_min = forward_propagation(X, W1, W2, b1, b2)
            cost_min = cost(Yhat_min, Y)
            dp[row, col] = (cost_plus - cost_min)/(2*epsilon)
            params[row, col] = backup
    return dp

## Bereken gradient op de trage manier
def gradient_check(X, Y, W1, W2, b1, b2, epsilon = 10**-6):    
    dW1 = compute_gradient_slow(W1, epsilon, X, Y, W1, W2, b1, b2)
    dW2 = compute_gradient_slow(W2, epsilon, X, Y, W1, W2, b1, b2)
    db1 = compute_gradient_slow(b1, epsilon, X, Y, W1, W2, b1, b2)
    db2 = compute_gradient_slow(b2, epsilon, X, Y, W1, W2, b1, b2)
    return dW1, dW2, db1, db2    


## Kleine check voor wat random waarden
np.random.seed(1)
(n0, n1, n2) = (5, 10, 7)
m = 100
X = np.random.randn(n0, m)
Y = np.random.randn(n2, m)
W1 = np.random.randn(n1, n0)
b1 = np.random.randn(n1,1)
W2 = np.random.randn(n2, n1)
b2 = np.random.randn(n2,1)

A1, A2 = forward_propagation(X, W1, W2, b1, b2)
dW1, dW2, db1, db2 = backward_propagation(X, Y, W1, W2, b1, b2, A1, A2)
dW1_check, dW2_check, db1_check, db2_check = gradient_check(X, Y, W1, W2, b1, b2)

assert np.linalg.norm(dW1 - dW1_check) < 10**-6, "Foute gradient voor W1"
assert np.linalg.norm(dW2 - dW2_check) < 10**-6, "Foute gradient voor W2"
assert np.linalg.norm(db1 - db1_check) < 10**-6, "Foute gradient voor b1"
assert np.linalg.norm(db2 - db2_check) < 10**-6, "Foute gradient voor b2"

Implementeer de code om de gewichtenmatrices en biasvectoren te initialiseren.
De gewichtenmatrices initialiseer je met kleine random waarden, 
gebruik bv. <code>np.random.randn(...)*0.01</code>. De biasvectoren mogen 
geïntialiseerd worden met allemaal nullen.

In [ ]:
def initialize(n0, n1, n2):
    """ Bepaal random gewichtenmatrices en biasvectoren van de juiste vorm.
    
        n0 : aantal neuronen in de invoerlaag
        n1 : aantal neuronen in de verborgen laag
        n2 : aantal neuronen in de uitvoerlaag
        
        Returns: W1, W2, b1, b2
    """
    ### BEGIN CODE HIER (4 regels)
  
    ### EINDE CODE HIER
    return W1, W2, b1, b2

In [ ]:
## TEST Cel: Niet wijzigen
(n0, n1, n2) = (4,7,5)
W1, W2, b1, b2 = initialize(n0, n1, n2)
assert W1.shape == (n1, n0), "W1 heeft verkeerde vorm"
assert W2.shape == (n2, n1), "W2 heeft verkeerde vorm"
assert b1.shape == (n1, 1), "b1 heeft verkeerde vorm"
assert b2.shape == (n2, 1), "b2 heeft verkeerde vorm"

We schrijven nu de code om één enkele update uit te voeren, aannemend dat we de 
aanpassingsmatrices en -vectoren reeds hebben berekend.


In [ ]:
def update(W1, W2, b1, b2, dW1, dW2, db1, db2, alpha):
    """ Voer één enkele update uit.
    
        W1 : eerste gewichten matrix van de vorm (n1, n0)
        W2 : tweede gewichten matrix van de vorm (n2, n1)
        b1 : eerste bias vector van de vorm (n1, 1)
        b2 : tweede bias vector van de vorm (n2, 1)
        dW1 : aanpassingsmatrix van de vorm (n1, n0)
        dW2 : aanpassingsmatrix van de vorm (n2, n1)
        db1 : aanpassingsvector van de vorm (n1, 1)
        db2 : aanpassingsvector van de vorm (n2, 1)
        alpha : de stapgrootte
        
        Returns: aangepaste versies van W1, W2, b1, b2
    """
    ### BEGIN CODE HIER (4 regels)
   
    ### EINDE CODE HIER
    return W1, W2, b1, b2

In [ ]:
## TEST Cel: Niet wijzigen
(n0, n1, n2) = (4,7,5)
np.random.seed(1)
W1, W2, b1, b2 = initialize(n0, n1, n2)
dW1, dW2, _, _ =initialize(n0, n1, n2)
db1 = np.random.randn(n1, 1)
db2 = np.random.randn(n2, 2)
alpha = 0.05
x, y, z, u = update(W1, W2, b1, b2, dW1, dW2, db1, db2, alpha)
W1bis, W2bis, b1bis, b2bis = update(x, y, z, u, dW1, dW2, db1, db2, -alpha)
assert np.linalg.norm(W1 - W1bis) < 10**-6, "W1 verkeerd"
assert np.linalg.norm(W2 - W2bis) < 10**-6, "W2 verkeerd"
assert np.linalg.norm(b1 - b1bis) < 10**-6, "b1 verkeerd"
assert np.linalg.norm(b2 - b2bis) < 10**-6, "b2 verkeerd"

Nu implementeren we het eigenlijke optimalisatieproces.

We delen de trainingsdata op in een minibatches. Per epoch overlopen we alle minibatches.
Voor elke minibatch roepen we <code>forward_propagation</code> aan op de activaties te berekenen.
Vervolgens roepen we <code>backward_propagation</code> en <code>update</code> aan.

Om te verifiëren of het trainingsproces correct werkt houden we ook de kost per epoch bij. Wanneer
het trainingsproces correct werkt moet deze kosthistoriek een dalende trend vertonen.

In [ ]:
def train(X, Y, W1, W2, b1, b2, num_epochs = 1000, mini_batch_size = 1, alpha = 0.05):
    """  Traint een neuraal netwerk en retourneert de getrainde gewichtenmatrices en biasvectoren.
    
        X  : Invoer voor het neuraal netwerk van de vorm (n0, m)
        Y  : Gewenste uitvoer van de vorm (n2, m)
        W1 : startwaarde eerste gewichtenmatrix, van de vorm (n1, n0)
        W2 : startwaarde tweede gewichtenmatrix, van de vorm (n2, n1)
        b1 : startwaarde eerste biasvector, van de vorm (n1, 1)
        b2 : startwaarde tweede biasvector, van de vorm (n2, 1)
        num_epochs : hoeveel keer gaan we over alle data
        mini_batch_size : hoeveel voorbeelden in één mini-batch ?
        alpha : stapgrootte van de updata
        
        returns:  W1, W2, b1, b2, costs  waarbij costs de historiek van de waarde van de kostfunctie voorstelt
    """
    (n, m) = X.shape
    num_batches = m // mini_batch_size + (0 if m % mini_batch_size == 0 else 1)
    costs = np.zeros(num_epochs)
    for epoch in range(num_epochs):
        cost_epoch = 0
        for batch in range(num_batches):
            ## Bepaal een mini-batch
            ## START CODE HIER (4 regels)
           
            ## EINDE CODE HIER
            ## Doe voorwaartse propagatie
            ## START CODE HIER (1 regel)
            
            ## EINDE CODE HIER 
            ## Doe achterwaartse propagatie
            ## START CODE HIER (1 regel)
            ## EINDE CODE HIER
            if False: ## zet op True op de gradiënt te debuggen --> Traag !
                dW1_check, dW2_check, db1_check, db2_check = gradient_check(Xbatch, Ybatch, W1, W2, b1, b2)
                assert np.linalg.norm(dW1 - dW1_check) < 10**-6, f"Foute gradient voor W1 {dW1} {dW1_check}"
                assert np.linalg.norm(dW2 - dW2_check) < 10**-6, "Foute gradient voor W2"
                assert np.linalg.norm(db1 - db1_check) < 10**-6, "Foute gradient voor b1"
                assert np.linalg.norm(db2 - db2_check) < 10**-6, "Foute gradient voor b2"
            ## Voer een update uit
            ## START CODE HIER (1 regel)
            ## EINDE CODE HIER
            # Bepaal kost           
            cost_epoch += (batch_end - batch_start + 1) * cost(A2,  Ybatch)
        cost_epoch /= m
        costs[epoch] = cost_epoch
    return W1, W2, b1, b2, costs            

## Testen op Iris Dataset

We testen ons neuraal netwerk op de klassieke "Iris" dataset.  Deze bevat gegevens over 150 irissen. Per iris worden er vier attributen gegeven (die te maken hebben met de afmetingen van de iris). Het doel is om te bepalen tot welk van 3 klassen een bepaalde iris behoort.

We zullen hiertoe een klein neuraal netwerk opbouwen en trainen m.b.v. de code die we hierboven hebben geïmplementeerd.

We laden de dataset in het geheugen en voeren enkele bewerkingen uit zodanig dat de data heeft die wordt verwacht door het neurale netwerk.

In [ ]:
from sklearn import datasets
from sklearn.preprocessing import OneHotEncoder
iris = datasets.load_iris()
print(f"Het type van iris.data is {type(iris.data)}")
print(f"Deze numpy array heeft de volgende vorm: {iris.data.shape}")

Het lijkt er m.a.w. op dat <code>iris.data</code> een observatie bevat per rij.  
Ons neuraal netwerk verwacht echter één observatie per kolom, i.e. de vorm van <code>X</code> is
<code>(n, m)</code>.

Schrijf de code om <code>iris.data</code> te transponeren.

In [ ]:
## BEGIN CODE HIER (1 regel)
## EINDE CODE HIER

In [ ]:
## TEST Cel: Niet wijzigen
assert X.shape == (4, 150), "Verkeerde vorm voor X"

Nu bekijken we de labels.

In [ ]:
print(f"Het type van iris.target is {type(iris.target)}")
print(f"De vorm van de numpy array is {iris.target.shape}")

De verwachte uitvoer <code>Y</code> moet van de vorm <code>(3,150)</code> zijn. Er zijn immers 3 klassen en 150 voorbeelden. We gebruiken een <code>OneHotEncoder</code>.

In [ ]:
encoder = OneHotEncoder(categories='auto', sparse=False)
Y = encoder.fit_transform(iris.target.reshape(-1,1))
print(f"Y.shape = {Y.shape}")
print("De eerste 5 rijen van Y zijn")
print(Y[:5,:])
print("De laatste 5 rijen van Y zijn")
print(Y[-5:,:])

De array <code>Y/<code> heeft nog steeds de verkeerde vorm. Transponeer <code>Y</code>.

In [ ]:
## BEGIN CODE HIER (1 regel)
## EINDE CODE HIER
assert Y.shape == (3,150), "Verkeerde vorm Y"

De datapunten zijn in dit geval zodanig geordend dat eerst alle voorbeelden van klasse 1 aan bod komen, dan alle voorbeelden van klasse 2 en tenslotte al de voorbeelden van klasse 3.  We permuteren de voorbeelden zodanig dat ze in een random volgorde voorkomen.

In [ ]:
permute = np.random.permutation(150)
X = X[:,permute]
Y = Y[:,permute]

We zetten een dertigtal voorbeelden apart om te kunnen nagaan of ook 'nieuwe' voorbeelden goed worden geclassificeerd.

In [ ]:
X_train, X_test = X[:,:120], X[:,120:]
Y_train, Y_test = Y[:,:120], Y[:,120:]

We bouwen nu een klein neuraal netwerk om dit classificatieprobleem aan te pakken.
Kies je parameters zodanig dat de verborgen laag 3 neuronen heeft.

We gebruiken bv. 1000 epochs met een mini batch grootte van 12.

In [ ]:
%%time 
## BEGIN CODE HIER

## EINDE CODE HIER

We plotten de kost. Als alles goed gaat dan zie je een sterk dalende curve bij de start van het trainingsproces. Rond de 500 epochs vlakt de kostfunctie af.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(cost_hist)
plt.show()

Bereken de kost op de trainingsdata. Bepaal hiervoor eerst de activaties voor de trainingsdata en roep vervolgens de kostfunctie aan.  Doe dan hetzelfde voor de testdata.

In [ ]:
A1, A2 = forward_propagation(X_train, W1, W2, b1, b2)
print(f"Kost op trainingsdata = {cost(A2, Y_train)}")
A1_test, A2_test = forward_propagation(X_test, W1, W2, b1, b2)
print(f"Kost op testdata = {cost(A2_test, Y_test)}")      

Het uiteindelijke doel is classificatie. We bekijken welk percentage van de voorbeelden er juist is geclassificeerd. We kiezen de uitvoerneuron met de hoogste waarde als uiteindelijke klasse.

In [ ]:
# Bepaal voor elke kolom van A2 de positie van het maximum
preds = np.argmax(A2, axis = 0)
actuals_train = np.argmax(Y_train, axis = 0)
print(f"Het percentage juiste geclassificeerde trainingsvoorbeelden {np.mean(preds == actuals_train)}")
preds_test = np.argmax(A2_test, axis = 0)
actuals_test = np.argmax(Y_test, axis = 0)
print(f"Het percentage juiste geclassificeerde testvoorbeelden {np.mean(preds_test == actuals_test)}")

We zien dat we hier een zeer goed accuraatheid bekomen zowel op de trainings- als testdata.

## MNIST Dataset

In [ ]:
from keras.datasets import mnist

We laden de dataset in en bekijken de vorm van de verschillende arrays.

In [ ]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
print('X_train.shape = ', X_train.shape)
print('Y_train.shape = ', Y_train.shape)
print('X_test.shape = ', X_test.shape)
print('Y_test.shape = ', Y_test.shape)

We bekijken enkele voorbeelden.

In [ ]:
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.axis('off')
    plt.imshow(X_train[i], cmap='gray', interpolation='none')
    plt.title("Class {}".format(Y_train[i]))

Er zijn maw 60000 training voorbeelden. Elk voorbeeld is een 28 bij 28 'matrix'. Die zullen we echter gebruiken als een ééndimensionale vector met 784 entries. De labels y zijn opgeslaan als getallen, en niet als een one-hot geëncodeerde vector.

Er zijn 10000 voorbeelden apart gehouden om te testen.

In [ ]:
X_train_flat = X_train.reshape(60000, 28 * 28)
X_test_flat = X_test.reshape(10000, 28 * 28)
print('Nieuwe shape =', X_train_flat.shape)

Aangezien we nu toch al keras gebruiken gebruiken we een keras functie om de one-hot encodering uit te voeren.


In [ ]:
import keras.utils
Y_train_one_hot = keras.utils.to_categorical(Y_train)
Y_test_one_hot = keras.utils.to_categorical(Y_test)
print('Nieuwe shape =', Y_train_one_hot.shape)

De matrices zijn nog altijd niet in de juiste vorm voor ons netwerk. Transponeer ze.


In [ ]:
### BEGIN CODE HIER (4 regels)

### EINDE CODE HIER

Het netwerk zal wellicht sneller kunnen getraind worden als we ervoor zorgen dat alle inputs tussen 0 en 1 liggen.

We herschalen de $X$-waarden door deze te delen door 255.

In [ ]:
### BEGIN CODE HIER

### EINDE CODE HIER

In [ ]:
## TEST CEL: niet wijzigen
assert X_train_flat.shape == (28 * 28, 60000), "Verkeerde vorm X_train_flat"
assert X_test_flat.shape == (28 * 28, 10000), "Verkeerde vorm X_test_flat"
assert Y_train_one_hot.shape == (10, 60000), "Verkeerde vorm Y_train_one_hot"
assert Y_test_one_hot.shape == (10, 10000), "Verkeerde vorm Y_test_one_hot"

We trachten nu ons netwerk te trainen. Bouw een netwerk op met 100 neuronen in de verborgen laag.  Train het netwerk.

In [ ]:
%%time 

## BEGIN CODE HIER

## EINDE CODE HIER

In [ ]:
plt.plot(cost_hist)
plt.show()

In [ ]:
## Accuraatheid
A1, A2 = forward_propagation(X_train_flat, W1, W2, b1, b2)
A1_test, A2_test = forward_propagation(X_test_flat, W1, W2, b1, b2)
preds = np.argmax(A2, axis = 0)
actuals_train = np.argmax(Y_train_one_hot, axis = 0)
print(f"Het percentage juiste geclassificeerde trainingsvoorbeelden {np.mean(preds == actuals_train)}")
preds_test = np.argmax(A2_test, axis = 0)
actuals_test = np.argmax(Y_test_one_hot, axis = 0)
print(f"Het percentage juiste geclassificeerde testvoorbeelden {np.mean(preds_test == actuals_test)}")